In [3]:
import pandas as pd

df = pd.read_csv("books.xls")
df.isnull().sum()

df=df.dropna()

df=df[["isbn13","title","authors","categories","description"]]

df.iloc[50:100,1:2]

def convert(text):
    L=[]
    for i in text.split():
        L.append(i)
    return L

df["authors"]=df["authors"].apply(convert)

df["authors"]=df["authors"].apply(lambda x:[i.replace(" ","") for i in x])

df["categories"]=df["categories"].apply(convert)

df["categories"]=df["categories"].apply(lambda x:[i.replace(" ","") for i in x])

df["description"]=df["description"].apply(convert)

df["description"]=df["description"].apply(lambda x:[i.replace(" ","") for i in x])

df["tags"]=df["authors"]+df["categories"]+df["description"]
df.head(1)

df["tags"]=df["tags"]
df.head(1)

final_df=df[["isbn13","title","tags"]]
final_df["tags"]=final_df["tags"].apply(lambda x:" ".join(x))

final_df["tags"]=final_df["tags"].str.lower()

final_df.head(1)

from nltk.stem import PorterStemmer

ps = PorterStemmer()

def stem(text):
    L=[]
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)

final_df["tags"]=final_df["tags"].apply(stem)

final_df["title"]=final_df["title"].astype("str")

final_df["title"]=final_df["title"].str.lower()

final_df.head(1)

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_features=5000,stop_words="english")

tokens=cv.fit_transform(final_df["tags"]).toarray()
simi=cosine_similarity(tokens)

def recommend (book_title):
    matched_book=final_df[final_df["title"]==book_title.lower()].index
    if len(matched_book)==0:
        return "not matched"
    matched_book=matched_book[0]
    simil=simi[matched_book]
    book_list=sorted(list(enumerate(simil)),reverse=True,key=lambda x:x[1])[1:6]
    print("recommed books")
    for i in book_list:
        print(final_df.iloc[i[0]]["title"])

recommend("gold")



C:\Users\kotha\AppData\Local\Temp\ipykernel_34104\3515257823.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["tags"]=final_df["tags"].apply(lambda x:" ".join(x))
C:\Users\kotha\AppData\Local\Temp\ipykernel_34104\3515257823.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["tags"]=final_df["tags"].str.lower()
C:\Users\kotha\AppData\Local\Temp\ipykernel_34104\3515257823.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

recommed books
history of religious ideas, volume 3
the visual arts
cuba
1968
the pursuit of history


In [4]:
import joblib

joblib.dump(final_df, "final_df.joblib")
joblib.dump(cv, "count_vectorizer.joblib")
joblib.dump(simi, "similarity_matrix.joblib")


['similarity_matrix.joblib']